# 🔍 SWaT 데이터셋 LSTM-AE 기반 이상 탐지 (Anomaly Detection)

---

## 📋 개요

| 항목 | 내용 |
|------|------|
| **데이터셋** | SWaT (Secure Water Treatment) - 싱가포르 iTrust 연구소 |
| **모델** | LSTM-AE (Long Short-Term Memory Autoencoder) |
| **방법** | 비지도 이상 탐지 - 정상 데이터로 학습 후 Reconstruction Error 기반 탐지 |
| **평가지표** | Precision, Recall, F1-Score, AUC-ROC |

---

## 📁 파이프라인

```
1. 환경 설정 및 라이브러리 임포트
2. 데이터 로딩
3. 데이터 전처리 (다운샘플링, 정규화, 시퀀스 생성)
4. LSTM-AE 모델 정의
5. 모델 학습
6. 이상 탐지 및 평가
7. 결과 시각화
```

---
## 1️⃣ 환경 설정 및 라이브러리 임포트

필요한 라이브러리들을 임포트하고 GPU 사용 가능 여부를 확인합니다.

In [1]:
# ============================================================
# 라이브러리 임포트
# ============================================================

# 데이터 처리
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리 및 평가
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, precision_recall_curve, roc_curve,
    confusion_matrix, classification_report
)

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 유틸리티
from tqdm.notebook import tqdm  # Jupyter용 프로그레스 바
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


In [2]:
# ============================================================
# GPU/CPU 디바이스 설정
# ============================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🖥️ 사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"   GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

🖥️ 사용 디바이스: cuda
   GPU 이름: NVIDIA GeForce RTX 3060 Laptop GPU
   GPU 메모리: 6.44 GB


---
## 2️⃣ 하이퍼파라미터 설정

전처리, 모델, 학습에 사용할 하이퍼파라미터를 정의합니다.

### 주요 파라미터 설명

| 파라미터 | 설명 | 권장값 |
|----------|------|--------|
| `downsample_rate` | 데이터 축소 비율 (10 = 1/10) | 10~20 |
| `window_size` | 시퀀스 길이 (타임스텝 수) | 50~200 |
| `stride` | 윈도우 이동 간격 | window_size의 10~20% |
| `hidden_dim` | LSTM 은닉층 차원 | 64~256 |
| `latent_dim` | 잠재 공간 차원 | 32~128 |
| `num_layers` | LSTM 레이어 수 | 1~3 |
| `threshold_percentile` | 이상 판정 임계값 백분위 | 95~99 |

In [ ]:
# ============================================================
# 하이퍼파라미터 설정
# ============================================================

CONFIG = {
    # ----- 데이터 경로 -----
    'normal_data_path': './data/normal.csv',
    'attack_data_path': './data/attack.csv',
    
    # ----- 전처리 파라미터 -----
    'downsample_rate': 10,        # 다운샘플링 비율 (1초 → 10초)
    'variance_threshold': 0.01,   # 저분산 컬럼 제거 임계값
    'correlation_threshold': 0.95, # 고상관 컬럼 제거 임계값
    
    # ----- 시퀀스 파라미터 -----
    'window_size': 100,  # 시퀀스 길이
    'stride': 10,        # 윈도우 이동 간격
    
    # ----- LSTM-AE 모델 파라미터 -----
    'hidden_dim': 128,    # LSTM 은닉층 차원
    'latent_dim': 64,     # 잠재 공간 차원
    'num_layers': 2,      # LSTM 레이어 수
    'dropout': 0.2,       # 드롭아웃 비율
    'bidirectional': False,  # 양방향 LSTM 사용 여부
    
    # ----- 학습 파라미터 -----
    'batch_size': 64,
    'learning_rate': 1e-3,
    'weight_decay': 1e-5,
    'epochs': 100,
    'patience': 10,  # Early Stopping
    
    # ----- 이상 탐지 파라미터 -----
    'threshold_percentile': 99.0  # 상위 1%를 이상으로 판정
}

# 설정 출력
print("⚙️ 하이퍼파라미터 설정:")
print("=" * 50)
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

⚙️ 하이퍼파라미터 설정:
  normal_data_path: /root/workspace/AEAD/Dataset/swat/normal.csv
  attack_data_path: /root/workspace/AEAD/Dataset/swat/attack.csv
  downsample_rate: 10
  variance_threshold: 0.01
  correlation_threshold: 0.95
  window_size: 100
  stride: 10
  hidden_dim: 128
  latent_dim: 64
  num_layers: 2
  dropout: 0.2
  bidirectional: False
  batch_size: 64
  learning_rate: 0.001
  weight_decay: 1e-05
  epochs: 100
  patience: 10
  threshold_percentile: 99.0


---
## 3️⃣ 데이터 로딩

SWaT 데이터셋을 로딩합니다.

### 데이터셋 구성

| 파일 | 설명 | 내용 |
|------|------|------|
| `normal.csv` | 정상 운영 데이터 | 7일간 정상 운영 기록 |
| `attack.csv` | 공격 포함 테스트 데이터 | 4일간 36개 공격 시나리오 포함 |

### 컬럼 구성 (51개 센서 + 레이블)
- **FIT**: Flow meter
- **LIT**: Level transmitter  
- **AIT**: Analyzer indicator transmitter
- **MV**: Motorized valve
- **P**: Pump
- **UV**: UV dechlorinator
- **PIT**: Pressure indicator transmitter
- **DPIT**: Differential pressure indicator transmitter

In [8]:
# ============================================================
# 데이터 로딩
# ============================================================

print("📂 데이터 로딩 중...")
print("=" * 50)

# 정상 데이터 로딩
normal_df = pd.read_csv(CONFIG['normal_data_path'], low_memory=False)
print(f"✅ 정상 데이터 로딩 완료: {normal_df.shape}")

# 공격 데이터 로딩
attack_df = pd.read_csv(CONFIG['attack_data_path'], low_memory=False)
print(f"✅ 공격 데이터 로딩 완료: {attack_df.shape}")

📂 데이터 로딩 중...
✅ 정상 데이터 로딩 완료: (1387098, 53)
✅ 공격 데이터 로딩 완료: (54621, 53)


In [9]:
# ============================================================
# 데이터 미리보기
# ============================================================

print("📊 정상 데이터 미리보기:")
display(normal_df.head())

print("\n📊 정상 데이터 기본 정보:")
print(normal_df.info())

📊 정상 데이터 미리보기:


,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2.0,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2.0,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2.0,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2.0,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2.0,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal



📊 정상 데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387098 entries, 0 to 1387097
Data columns (total 53 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0    Timestamp     1387098 non-null  object 
 1   FIT101         1387098 non-null  float64
 2   LIT101         1387098 non-null  float64
 3    MV101         395298 non-null   float64
 4   P101           1387098 non-null  int64  
 5   P102           1387098 non-null  int64  
 6    AIT201        395298 non-null   float64
 7   AIT202         1387098 non-null  float64
 8   AIT203         1387098 non-null  float64
 9   FIT201         1387098 non-null  float64
 10   MV201         395298 non-null   float64
 11   P201          395298 non-null   float64
 12   P202          395298 non-null   float64
 13  P203           1387098 non-null  int64  
 14   P204          395298 non-null   float64
 15  P205           1387098 non-null  int64  
 16  P206           1387098 non-null  int6

In [10]:
# ============================================================
# 컬럼명 정리 및 레이블 추출 (SWaT 데이터셋용)
# ============================================================

# 컬럼명 정리 (공백 제거)
normal_df.columns = normal_df.columns.str.strip()
attack_df.columns = attack_df.columns.str.strip()

# 레이블 컬럼명 확인
label_col = 'Normal/Attack'

# 공격 레이블 추출 ('Normal' → 0, 'Attack' → 1)
attack_labels_raw = attack_df[label_col].str.strip().values
attack_labels = (attack_labels_raw == 'Attack').astype(int)

# 레이블 컬럼 제거
normal_df = normal_df.drop(columns=[label_col])
attack_df = attack_df.drop(columns=[label_col])

print(f"✅ 레이블 추출 완료")
print(f"   - 정상 (0): {np.sum(attack_labels == 0)}")
print(f"   - 공격 (1): {np.sum(attack_labels == 1)}")
print(f"   - 공격 비율: {np.mean(attack_labels) * 100:.2f}%")

✅ 레이블 추출 완료
   - 정상 (0): 0
   - 공격 (1): 54621
   - 공격 비율: 100.00%


---
## 4️⃣ 데이터 전처리

### 전처리 단계

1. **수치형 컬럼 선택**: Timestamp 등 비수치형 제거
2. **다운샘플링**: 데이터 크기 축소 (1초 → 10초 간격)
3. **결측치 처리**: 전방/후방 채움 + 평균값 대체
4. **저분산 컬럼 제거**: 분산이 거의 0인 컬럼 제거
5. **고상관 컬럼 제거**: 상관계수 0.95 이상인 중복 컬럼 제거
6. **정규화**: MinMax Scaling (0-1)

In [12]:
# ============================================================
# Step 1: 수치형 컬럼만 선택
# ============================================================

# 제외할 컬럼 (비수치형) - SWaT 데이터셋용
exclude_cols = ['Timestamp']
numeric_cols = [col for col in normal_df.columns if col not in exclude_cols]

# 수치형 데이터 추출
normal_numeric = normal_df[numeric_cols].copy()
attack_numeric = attack_df[[col for col in numeric_cols if col in attack_df.columns]].copy()

# 강제 수치형 변환 (문자열 → 숫자, 변환 불가 시 NaN)
normal_numeric = normal_numeric.apply(pd.to_numeric, errors='coerce')
attack_numeric = attack_numeric.apply(pd.to_numeric, errors='coerce')

print(f"✅ 수치형 컬럼 선택 완료: {len(numeric_cols)}개 컬럼")
print(f"   - 정상 데이터: {normal_numeric.shape}")
print(f"   - 공격 데이터: {attack_numeric.shape}")

✅ 수치형 컬럼 선택 완료: 51개 컬럼
   - 정상 데이터: (1387098, 51)
   - 공격 데이터: (54621, 51)


In [13]:
# ============================================================
# Step 2: 다운샘플링
# ============================================================

downsample_rate = CONFIG['downsample_rate']

# 매 downsample_rate 번째 행만 선택
normal_numeric = normal_numeric.iloc[::downsample_rate].reset_index(drop=True)
attack_numeric = attack_numeric.iloc[::downsample_rate].reset_index(drop=True)

# 레이블도 동일하게 다운샘플링
attack_labels = attack_labels[::downsample_rate]

print(f"✅ 다운샘플링 완료 (1/{downsample_rate})")
print(f"   - 정상 데이터: {normal_numeric.shape}")
print(f"   - 공격 데이터: {attack_numeric.shape}")

✅ 다운샘플링 완료 (1/10)
   - 정상 데이터: (138710, 51)
   - 공격 데이터: (5463, 51)


In [14]:
# ============================================================
# Step 3: 결측치 처리
# ============================================================

# 결측치 현황 확인
print("📊 결측치 현황:")
print(f"   - 정상 데이터: {normal_numeric.isna().sum().sum()}개")
print(f"   - 공격 데이터: {attack_numeric.isna().sum().sum()}개")

# 전방/후방 채움
normal_numeric = normal_numeric.ffill().bfill()
attack_numeric = attack_numeric.ffill().bfill()

# 남은 NaN은 컬럼 평균으로 대체
normal_numeric = normal_numeric.fillna(normal_numeric.mean())
attack_numeric = attack_numeric.fillna(attack_numeric.mean())

# 전체가 NaN인 컬럼 제거
valid_cols = normal_numeric.columns[~normal_numeric.isna().all()]
normal_numeric = normal_numeric[valid_cols]
attack_numeric = attack_numeric[[col for col in valid_cols if col in attack_numeric.columns]]

print(f"\n✅ 결측치 처리 완료")
print(f"   - 유효 컬럼 수: {len(valid_cols)}")

📊 결측치 현황:
   - 정상 데이터: 694260개
   - 공격 데이터: 0개

✅ 결측치 처리 완료
   - 유효 컬럼 수: 51


In [15]:
# ============================================================
# Step 4: 저분산 컬럼 제거
# ============================================================

# 각 컬럼의 분산 계산
variances = normal_numeric.var()

# 분산이 임계값 이상인 컬럼만 선택
high_var_cols = variances[variances > CONFIG['variance_threshold']].index.tolist()

# 제거된 컬럼 확인
removed_cols = set(normal_numeric.columns) - set(high_var_cols)
print(f"📊 저분산 컬럼 제거:")
print(f"   - 제거된 컬럼 수: {len(removed_cols)}")
if len(removed_cols) > 0 and len(removed_cols) <= 10:
    print(f"   - 제거된 컬럼: {list(removed_cols)}")

# 적용
normal_numeric = normal_numeric[high_var_cols]
attack_numeric = attack_numeric[[col for col in high_var_cols if col in attack_numeric.columns]]

print(f"\n✅ 저분산 컬럼 제거 완료: {len(high_var_cols)}개 컬럼 유지")

📊 저분산 컬럼 제거:
   - 제거된 컬럼 수: 20

✅ 저분산 컬럼 제거 완료: 31개 컬럼 유지


In [16]:
# ============================================================
# Step 5: 고상관 컬럼 제거 (선택적)
# ============================================================

if CONFIG['correlation_threshold'] < 1.0:
    # 상관 행렬 계산
    corr_matrix = normal_numeric.corr().abs()
    
    # 상삼각 행렬만 사용 (중복 제거)
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    
    # 임계값 이상의 상관관계를 가진 컬럼 찾기
    to_drop = [col for col in upper.columns 
               if any(upper[col] > CONFIG['correlation_threshold'])]
    
    print(f"📊 고상관 컬럼 제거:")
    print(f"   - 제거된 컬럼 수: {len(to_drop)}")
    
    # 적용
    normal_numeric = normal_numeric.drop(columns=to_drop)
    attack_numeric = attack_numeric.drop(
        columns=[col for col in to_drop if col in attack_numeric.columns]
    )
    
    print(f"✅ 고상관 컬럼 제거 완료: {normal_numeric.shape[1]}개 컬럼 유지")
else:
    print("⏭️ 고상관 컬럼 제거 건너뜀 (threshold=1.0)")

📊 고상관 컬럼 제거:
   - 제거된 컬럼 수: 7
✅ 고상관 컬럼 제거 완료: 24개 컬럼 유지


In [18]:
# ============================================================
# Step 6: 정규화 (MinMax Scaling)
# ============================================================

# 정상 데이터로 스케일러 학습
scaler = MinMaxScaler()
normal_scaled = scaler.fit_transform(normal_numeric)

# 공격 데이터에 동일한 스케일러 적용
attack_scaled = scaler.transform(attack_numeric)

# 피처 컬럼명 저장
feature_columns = normal_numeric.columns.tolist()
n_features = len(feature_columns)

print(f"✅ 정규화 완료 (MinMax Scaling)")
print(f"\n📊 최종 데이터 크기:")
print(f"   - 정상 데이터: {normal_scaled.shape}")
print(f"   - 공격 데이터: {attack_scaled.shape}")
print(f"   - 피처 수: {n_features}")

✅ 정규화 완료 (MinMax Scaling)

📊 최종 데이터 크기:
   - 정상 데이터: (138710, 24)
   - 공격 데이터: (5463, 24)
   - 피처 수: 24


---
## 5️⃣ 시퀀스 생성 (Sliding Window)

시계열 데이터를 LSTM-AE 모델에 입력하기 위해 슬라이딩 윈도우 방식으로 시퀀스를 생성합니다.

### 시퀀스 생성 방식

```
원본: [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, ...]

window_size=4, stride=2 인 경우:
  시퀀스 1: [x1, x2, x3, x4]
  시퀀스 2: [x3, x4, x5, x6]
  시퀀스 3: [x5, x6, x7, x8]
  ...
```

In [19]:
# ============================================================
# 시퀀스 생성 함수
# ============================================================

def create_sequences(data, window_size, stride):
    """
    슬라이딩 윈도우로 시퀀스 생성
    
    Args:
        data: 입력 데이터 (samples, features)
        window_size: 윈도우 크기 (시퀀스 길이)
        stride: 윈도우 이동 간격
        
    Returns:
        sequences: (num_sequences, window_size, features)
    """
    sequences = []
    for i in range(0, len(data) - window_size + 1, stride):
        seq = data[i:i + window_size]
        sequences.append(seq)
    
    return np.array(sequences)

print("✅ 시퀀스 생성 함수 정의 완료")

✅ 시퀀스 생성 함수 정의 완료


In [20]:
# ============================================================
# 학습/검증/테스트 데이터 분할 및 시퀀스 생성
# ============================================================

window_size = CONFIG['window_size']
stride = CONFIG['stride']

print(f"📊 시퀀스 생성 설정:")
print(f"   - Window Size: {window_size}")
print(f"   - Stride: {stride}")

# ------------------------------------------------------------
# 정상 데이터에서 미리 테스트용 구간을 떼어내기
#  - normal_test: attack 데이터와 동일한 길이만큼 사용 (클래스 비율 1:1 가정)
#  - 나머지(normal_train_val)를 다시 train/val로 분할
# ------------------------------------------------------------
num_attack_samples = len(attack_scaled)

if num_attack_samples >= len(normal_scaled):
    raise ValueError(
        f"공격 샘플 수({num_attack_samples})가 정상 샘플 수({len(normal_scaled)})보다 크거나 같습니다. " 
        "normal_test를 만들 수 없습니다. downsample_rate 또는 데이터 길이를 확인해주세요."
    )

# 뒤에서부터 공격 샘플 수와 동일한 길이만큼을 테스트용 정상 데이터로 사용
normal_test = normal_scaled[-num_attack_samples:]
normal_test_labels = np.zeros(len(normal_test), dtype=int)  # 모두 정상(0) 라벨

# 나머지는 학습/검증용으로 사용
normal_train_val = normal_scaled[:-num_attack_samples]

train_size = int(len(normal_train_val) * 0.8)
train_data = normal_train_val[:train_size]
val_data = normal_train_val[train_size:]

print(f"\n📊 데이터 분할:")
print(f"   - 전체 정상 데이터: {normal_scaled.shape}")
print(f"   - 학습용 정상 데이터: {train_data.shape}")
print(f"   - 검증용 정상 데이터: {val_data.shape}")
print(f"   - 테스트용 정상 데이터: {normal_test.shape}")
print(f"   - 공격 데이터(테스트용): {attack_scaled.shape}")

# ------------------------------------------------------------
# 시퀀스 생성
# ------------------------------------------------------------
print("\n⏳ 시퀀스 생성 중...")

train_sequences = create_sequences(train_data, window_size, stride)
val_sequences = create_sequences(val_data, window_size, stride)

# 테스트 데이터는 [정상 test + 공격]을 시간 순서대로 이어붙여서 구성
test_data = np.concatenate([normal_test, attack_scaled], axis=0)

# 시퀀스 생성
test_sequences = create_sequences(test_data, window_size, stride)

print(f"\n✅ 시퀀스 생성 완료:")
print(f"   - 학습: {train_sequences.shape}")
print(f"   - 검증: {val_sequences.shape}")
print(f"   - 테스트: {test_sequences.shape}")


📊 시퀀스 생성 설정:
   - Window Size: 100
   - Stride: 10

📊 데이터 분할:
   - 학습 데이터: (110968, 24)
   - 검증 데이터: (27742, 24)

⏳ 시퀀스 생성 중...

✅ 시퀀스 생성 완료:
   - 학습: (11087, 100, 24)
   - 검증: (2765, 100, 24)
   - 테스트: (537, 100, 24)


In [21]:
# ============================================================
# 테스트 레이블 생성 (윈도우 단위)
# ============================================================

# ------------------------------------------------------------
# 원시 시계열 단위 라벨 생성
#  - normal_test 구간: 모두 정상(0)
#  - attack 구간: 모두 공격(1)로 가정
#    (SWaT attack.csv에는 비정상 구간만 포함되어 있다고 가정)
# ------------------------------------------------------------
normal_test_labels = np.zeros(len(normal_test), dtype=int)
attack_test_labels = np.ones(len(attack_scaled), dtype=int)

test_labels_raw = np.concatenate([normal_test_labels, attack_test_labels])

# ------------------------------------------------------------
# 윈도우 단위 라벨 생성
#  - 하나의 윈도우(window_size 길이)의 평균 라벨이 0.5 이상이면 공격(1)
# ------------------------------------------------------------
test_labels = []
for i in range(0, len(test_labels_raw) - window_size + 1, stride):
    window_labels = test_labels_raw[i:i + window_size]
    test_labels.append(int(np.mean(window_labels) >= 0.5))

test_labels = np.array(test_labels)

print(f"✅ 테스트 레이블 생성 완료:")
print(f"   - 전체 시퀀스: {len(test_labels)}")
print(f"   - 정상 시퀀스: {np.sum(test_labels == 0)}")
print(f"   - 공격 시퀀스: {np.sum(test_labels == 1)}")
print(f"   - 공격 비율: {np.mean(test_labels) * 100:.2f}%")


✅ 테스트 레이블 생성 완료:
   - 전체 시퀀스: 537
   - 정상 시퀀스: 0
   - 공격 시퀀스: 537
   - 공격 비율: 100.00%


In [22]:
# ============================================================
# PyTorch DataLoader 생성
# ============================================================

# Tensor 변환
train_tensor = torch.FloatTensor(train_sequences)
val_tensor = torch.FloatTensor(val_sequences)
test_tensor = torch.FloatTensor(test_sequences)

# DataLoader 생성
batch_size = CONFIG['batch_size']

train_loader = DataLoader(
    TensorDataset(train_tensor),
    batch_size=batch_size,
    shuffle=True
)

val_loader = DataLoader(
    TensorDataset(val_tensor),
    batch_size=batch_size,
    shuffle=False
)

test_loader = DataLoader(
    TensorDataset(test_tensor),
    batch_size=batch_size,
    shuffle=False
)

print(f"✅ DataLoader 생성 완료")
print(f"   - Batch Size: {batch_size}")
print(f"   - 학습 배치 수: {len(train_loader)}")
print(f"   - 검증 배치 수: {len(val_loader)}")
print(f"   - 테스트 배치 수: {len(test_loader)}")

✅ DataLoader 생성 완료
   - Batch Size: 64
   - 학습 배치 수: 174
   - 검증 배치 수: 44
   - 테스트 배치 수: 9


---
## 6️⃣ LSTM-AE 모델 정의

### LSTM (Long Short-Term Memory) 특징

1. **장기 의존성 학습**: 게이트 메커니즘으로 장기 시퀀스 패턴 학습
2. **Forget Gate**: 불필요한 정보 삭제
3. **Input Gate**: 새로운 정보 추가
4. **Output Gate**: 출력할 정보 결정

### 모델 구조

```
Input (batch, seq_len, features)
    ↓
Encoder LSTM: features → hidden_dim → latent_dim
    ↓
Latent Vector (batch, latent_dim)
    ↓
Repeat Vector (batch, seq_len, latent_dim)
    ↓
Decoder LSTM: latent_dim → hidden_dim → features
    ↓
Output (batch, seq_len, features) - Reconstructed
```

In [23]:
# ============================================================
# LSTM Encoder
# ============================================================

class LSTMEncoder(nn.Module):
    """
    LSTM Encoder
    
    - 시퀀스를 잠재 공간으로 압축
    - 마지막 타임스텝의 hidden state를 잠재 벡터로 사용
    
    Args:
        input_dim: 입력 특성 수
        hidden_dim: LSTM 은닉층 차원
        latent_dim: 잠재 공간 차원
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
        bidirectional: 양방향 LSTM 사용 여부
    """
    
    def __init__(self, input_dim, hidden_dim=128, latent_dim=64, 
                 num_layers=2, dropout=0.2, bidirectional=False):
        super(LSTMEncoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1
        
        # LSTM 레이어
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        
        # Hidden state → Latent vector
        self.fc = nn.Linear(hidden_dim * self.num_directions, latent_dim)
        
    def forward(self, x):
        """
        Args:
            x: (batch, seq_len, input_dim)
        Returns:
            latent: (batch, latent_dim)
        """
        batch_size = x.size(0)
        
        # LSTM forward
        # output: (batch, seq_len, hidden_dim * num_directions)
        # h_n: (num_layers * num_directions, batch, hidden_dim)
        output, (h_n, c_n) = self.lstm(x)
        
        # 마지막 레이어의 hidden state 사용
        if self.bidirectional:
            # 양방향의 마지막 hidden state 연결
            h_forward = h_n[-2]  # 마지막 레이어 forward
            h_backward = h_n[-1]  # 마지막 레이어 backward
            h_last = torch.cat([h_forward, h_backward], dim=1)
        else:
            h_last = h_n[-1]  # (batch, hidden_dim)
        
        # Latent vector 생성
        latent = self.fc(h_last)  # (batch, latent_dim)
        
        return latent

print("✅ LSTMEncoder 정의 완료")

✅ LSTMEncoder 정의 완료


In [24]:
# ============================================================
# LSTM Decoder
# ============================================================

class LSTMDecoder(nn.Module):
    """
    LSTM Decoder
    
    - 잠재 벡터로부터 원본 시퀀스 복원
    - RepeatVector로 잠재 벡터를 시퀀스 길이만큼 반복
    
    Args:
        latent_dim: 잠재 공간 차원
        hidden_dim: LSTM 은닉층 차원
        output_dim: 출력 특성 수 (= 입력 특성 수)
        seq_len: 시퀀스 길이
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
    """
    
    def __init__(self, latent_dim, hidden_dim=128, output_dim=50, 
                 seq_len=100, num_layers=2, dropout=0.2):
        super(LSTMDecoder, self).__init__()
        
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.seq_len = seq_len
        self.num_layers = num_layers
        
        # Latent → Hidden 변환
        self.fc_in = nn.Linear(latent_dim, hidden_dim)
        
        # LSTM 레이어
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Output layer
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, latent):
        """
        Args:
            latent: (batch, latent_dim)
        Returns:
            output: (batch, seq_len, output_dim)
        """
        batch_size = latent.size(0)
        
        # Latent → Hidden 변환
        hidden_in = self.fc_in(latent)  # (batch, hidden_dim)
        
        # RepeatVector: 잠재 벡터를 seq_len만큼 반복
        repeated = hidden_in.unsqueeze(1).repeat(1, self.seq_len, 1)  # (batch, seq_len, hidden_dim)
        
        # LSTM forward
        lstm_out, _ = self.lstm(repeated)  # (batch, seq_len, hidden_dim)
        
        # Output 생성
        output = self.fc_out(lstm_out)  # (batch, seq_len, output_dim)
        
        return output

print("✅ LSTMDecoder 정의 완료")

✅ LSTMDecoder 정의 완료


In [25]:
# ============================================================
# LSTM Autoencoder (LSTM-AE)
# ============================================================

class LSTMAE(nn.Module):
    """
    LSTM Autoencoder
    
    이상 탐지 원리:
    - 정상 데이터로만 학습 → 정상 패턴 재구성 능력 학습
    - 이상 데이터는 재구성 오류(reconstruction error)가 높음
    
    Args:
        input_dim: 입력 특성 수
        hidden_dim: LSTM 은닉층 차원
        latent_dim: 잠재 공간 차원
        seq_len: 시퀀스 길이
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
        bidirectional: 인코더 양방향 LSTM 사용 여부
    """
    
    def __init__(self, input_dim, hidden_dim=128, latent_dim=64, 
                 seq_len=100, num_layers=2, dropout=0.2, bidirectional=False):
        super(LSTMAE, self).__init__()
        
        self.encoder = LSTMEncoder(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            latent_dim=latent_dim,
            num_layers=num_layers,
            dropout=dropout,
            bidirectional=bidirectional
        )
        
        self.decoder = LSTMDecoder(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            output_dim=input_dim,
            seq_len=seq_len,
            num_layers=num_layers,
            dropout=dropout
        )
        
    def forward(self, x):
        """
        Args:
            x: 입력 시퀀스 (batch, seq_len, features)
        Returns:
            reconstructed: 재구성된 시퀀스 (batch, seq_len, features)
        """
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed
    
    def get_reconstruction_error(self, x):
        """재구성 오류 계산"""
        self.eval()
        with torch.no_grad():
            reconstructed = self.forward(x)
            # MSE per sample
            error = torch.mean((x - reconstructed) ** 2, dim=(1, 2))
        return error
    
    def get_latent(self, x):
        """잠재 벡터 추출"""
        self.eval()
        with torch.no_grad():
            latent = self.encoder(x)
        return latent

print("✅ LSTMAE 정의 완료")

✅ LSTMAE 정의 완료


In [26]:
# ============================================================
# 모델 인스턴스 생성
# ============================================================

model = LSTMAE(
    input_dim=n_features,
    hidden_dim=CONFIG['hidden_dim'],
    latent_dim=CONFIG['latent_dim'],
    seq_len=CONFIG['window_size'],
    num_layers=CONFIG['num_layers'],
    dropout=CONFIG['dropout'],
    bidirectional=CONFIG['bidirectional']
).to(device)

# 모델 구조 출력
print("🏗️ 모델 구조:")
print("=" * 60)
print(model)

# 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 파라미터 수:")
print(f"   - 전체: {total_params:,}")
print(f"   - 학습 가능: {trainable_params:,}")

🏗️ 모델 구조:
LSTMAE(
  (encoder): LSTMEncoder(
    (lstm): LSTM(24, 128, num_layers=2, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): LSTMDecoder(
    (fc_in): Linear(in_features=64, out_features=128, bias=True)
    (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.2)
    (fc_out): Linear(in_features=128, out_features=24, bias=True)
  )
)

📊 파라미터 수:
   - 전체: 494,808
   - 학습 가능: 494,808


---
## 7️⃣ 모델 학습

### 학습 전략

- **손실 함수**: MSE (Mean Squared Error) - 재구성 손실
- **옵티마이저**: Adam with weight decay (L2 정규화)
- **Early Stopping**: 검증 손실이 개선되지 않으면 조기 종료
- **Gradient Clipping**: 그래디언트 폭발 방지 (LSTM에서 특히 중요)

In [27]:
# ============================================================
# 학습 설정
# ============================================================

criterion = nn.MSELoss()  # 재구성 손실
optimizer = optim.Adam(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# 학습 기록
train_losses = []
val_losses = []

print("✅ 학습 설정 완료")
print(f"   - 손실 함수: MSELoss")
print(f"   - 옵티마이저: Adam (lr={CONFIG['learning_rate']}, weight_decay={CONFIG['weight_decay']})")

✅ 학습 설정 완료
   - 손실 함수: MSELoss
   - 옵티마이저: Adam (lr=0.001, weight_decay=1e-05)


In [28]:
# ============================================================
# 학습 함수 정의
# ============================================================

def train_epoch(model, dataloader, criterion, optimizer, device):
    """한 에폭 학습"""
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        x = batch[0].to(device)
        
        # Forward
        reconstructed = model(x)
        loss = criterion(reconstructed, x)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Clipping (LSTM에서 중요)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(dataloader)


def validate(model, dataloader, criterion, device):
    """검증 손실 계산"""
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            x = batch[0].to(device)
            reconstructed = model(x)
            loss = criterion(reconstructed, x)
            total_loss += loss.item()
    
    return total_loss / len(dataloader)

print("✅ 학습/검증 함수 정의 완료")

✅ 학습/검증 함수 정의 완료


In [ ]:
# ============================================================
# 모델 학습 실행
# ============================================================

print("🚀 모델 학습 시작!")
print("=" * 60)

epochs = CONFIG['epochs']
patience = CONFIG['patience']

best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

# 학습 루프
for epoch in tqdm(range(epochs), desc="Training"):
    # 학습
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # 검증
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
    
    # 진행 상황 출력 (매 10 에폭)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.6f}, "
              f"Val Loss: {val_loss:.6f}")
    
    # Early Stopping
    if patience_counter >= patience:
        print(f"\n⏹️ Early Stopping at epoch {epoch+1}")
        break

# Best 모델 복원
if best_model_state is not None:
    model.load_state_dict(best_model_state)

print(f"\n✅ 학습 완료!")
print(f"   - Best Validation Loss: {best_val_loss:.6f}")

🚀 모델 학습 시작!


Training:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# ============================================================
# 학습 곡선 시각화
# ============================================================

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue', linewidth=2)
plt.plot(val_losses, label='Validation Loss', color='orange', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Learning Curve')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss', color='blue', linewidth=2)
plt.plot(val_losses, label='Validation Loss', color='orange', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss (Log Scale)')
plt.title('Learning Curve (Log Scale)')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('learning_curve_lstm.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 학습 곡선이 'learning_curve_lstm.png'로 저장되었습니다.")

---
## 8️⃣ 이상 탐지 및 평가

### 이상 탐지 원리

1. 정상 데이터로 학습된 모델은 **정상 패턴**을 잘 재구성함
2. **이상 데이터**는 학습하지 않은 패턴이므로 재구성 오류가 높음
3. 재구성 오류가 **임계값(threshold)** 이상이면 이상으로 판정

### 임계값 결정

검증 데이터(정상)의 재구성 오류 분포에서 상위 1% (99 percentile)를 임계값으로 설정

In [ ]:
# ============================================================
# 이상 점수 (Reconstruction Error) 계산 함수
# ============================================================

def compute_anomaly_scores(model, dataloader, device):
    """
    이상 점수 계산
    
    Args:
        model: 학습된 LSTM-AE 모델
        dataloader: 데이터 로더
        device: 디바이스
        
    Returns:
        scores: 각 샘플의 재구성 오류 (이상 점수)
    """
    model.eval()
    all_scores = []
    
    with torch.no_grad():
        for batch in dataloader:
            x = batch[0].to(device)
            scores = model.get_reconstruction_error(x)
            all_scores.extend(scores.cpu().numpy())
    
    return np.array(all_scores)

print("✅ 이상 점수 계산 함수 정의 완료")

In [ ]:
# ============================================================
# 이상 점수 계산
# ============================================================

print("⏳ 이상 점수 계산 중...")

# 검증 데이터 (정상)의 이상 점수
val_scores = compute_anomaly_scores(model, val_loader, device)

# 테스트 데이터의 이상 점수
test_scores = compute_anomaly_scores(model, test_loader, device)

print(f"\n✅ 이상 점수 계산 완료")
print(f"   - 검증 데이터 점수: mean={np.mean(val_scores):.6f}, std={np.std(val_scores):.6f}")
print(f"   - 테스트 데이터 점수: mean={np.mean(test_scores):.6f}, std={np.std(test_scores):.6f}")

In [ ]:
# ============================================================
# 임계값 결정
# ============================================================

# 검증 데이터(정상)의 상위 percentile을 임계값으로 설정
threshold = np.percentile(val_scores, CONFIG['threshold_percentile'])

print(f"📊 임계값 결정:")
print(f"   - Percentile: {CONFIG['threshold_percentile']}%")
print(f"   - Threshold: {threshold:.6f}")

# 다양한 percentile 값 확인
print(f"\n   참고 - 다른 percentile 값:")
for p in [90, 95, 99, 99.5]:
    print(f"   - {p}%: {np.percentile(val_scores, p):.6f}")

In [ ]:
# ============================================================
# 예측 및 성능 평가
# ============================================================

# 예측 (점수가 임계값 이상이면 이상으로 판정)
predictions = (test_scores >= threshold).astype(int)

# 성능 지표 계산
precision = precision_score(test_labels, predictions, zero_division=0)
recall = recall_score(test_labels, predictions, zero_division=0)
f1 = f1_score(test_labels, predictions, zero_division=0)

try:
    auc_roc = roc_auc_score(test_labels, test_scores)
except:
    auc_roc = 0.0

# Confusion Matrix
cm = confusion_matrix(test_labels, predictions)

print("=" * 60)
print("📊 이상 탐지 성능 평가 결과")
print("=" * 60)
print(f"\n🎯 주요 지표:")
print(f"   - Precision (정밀도): {precision:.4f}")
print(f"   - Recall (재현율):    {recall:.4f}")
print(f"   - F1-Score:           {f1:.4f}")
print(f"   - AUC-ROC:            {auc_roc:.4f}")

print(f"\n📋 Confusion Matrix:")
print(f"                 Predicted")
print(f"                 Normal  Attack")
print(f"Actual Normal    {cm[0,0]:6d}  {cm[0,1]:6d}")
print(f"Actual Attack    {cm[1,0]:6d}  {cm[1,1]:6d}")

In [ ]:
# ============================================================
# 상세 분류 보고서
# ============================================================

print("\n📋 Classification Report:")
print("=" * 60)
print(classification_report(test_labels, predictions, target_names=['Normal', 'Attack']))

---
## 9️⃣ 결과 시각화

### 시각화 항목

1. **이상 점수 분포**: 정상 vs 공격 데이터의 점수 분포
2. **ROC Curve**: True Positive Rate vs False Positive Rate
3. **Precision-Recall Curve**: 정밀도-재현율 곡선
4. **시간별 이상 점수**: 테스트 기간 동안의 이상 점수 변화

In [ ]:
# ============================================================
# 결과 시각화
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ----- (1) 이상 점수 분포 -----
ax1 = axes[0, 0]
ax1.hist(val_scores, bins=50, alpha=0.7, label='Validation (Normal)', color='blue', density=True)
ax1.hist(test_scores[test_labels == 0], bins=50, alpha=0.5, label='Test Normal', color='green', density=True)
ax1.hist(test_scores[test_labels == 1], bins=50, alpha=0.5, label='Test Attack', color='red', density=True)
ax1.axvline(threshold, color='black', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.4f}')
ax1.set_xlabel('Reconstruction Error (Anomaly Score)')
ax1.set_ylabel('Density')
ax1.set_title('Anomaly Score Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# ----- (2) ROC Curve -----
ax2 = axes[0, 1]
try:
    fpr, tpr, _ = roc_curve(test_labels, test_scores)
    ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {auc_roc:.4f})')
    ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('ROC Curve')
    ax2.legend(loc='lower right')
    ax2.grid(True, alpha=0.3)
except:
    ax2.text(0.5, 0.5, 'ROC Curve unavailable', ha='center', va='center')

# ----- (3) Precision-Recall Curve -----
ax3 = axes[1, 0]
try:
    precision_curve, recall_curve, _ = precision_recall_curve(test_labels, test_scores)
    ax3.plot(recall_curve, precision_curve, color='green', lw=2)
    ax3.set_xlabel('Recall')
    ax3.set_ylabel('Precision')
    ax3.set_title('Precision-Recall Curve')
    ax3.grid(True, alpha=0.3)
except:
    ax3.text(0.5, 0.5, 'PR Curve unavailable', ha='center', va='center')

# ----- (4) 시간에 따른 이상 점수 -----
ax4 = axes[1, 1]
ax4.plot(test_scores, alpha=0.7, linewidth=0.5, label='Anomaly Score', color='blue')
ax4.axhline(threshold, color='red', linestyle='--', linewidth=2, label='Threshold')

# 실제 공격 구간 표시
attack_indices = np.where(test_labels == 1)[0]
if len(attack_indices) > 0:
    ax4.scatter(attack_indices, test_scores[attack_indices], 
               c='red', s=1, alpha=0.3, label='Attack Points')

ax4.set_xlabel('Sample Index')
ax4.set_ylabel('Anomaly Score')
ax4.set_title('Anomaly Scores Over Time')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('anomaly_detection_results_lstm.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 결과 그래프가 'anomaly_detection_results_lstm.png'로 저장되었습니다.")

In [ ]:
# ============================================================
# Confusion Matrix 히트맵
# ============================================================

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal', 'Attack'],
            yticklabels=['Normal', 'Attack'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (LSTM-AE)')
plt.tight_layout()
plt.savefig('confusion_matrix_lstm.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 Confusion Matrix가 'confusion_matrix_lstm.png'로 저장되었습니다.")

---
## 🔟 모델 저장

학습된 모델과 설정을 저장합니다.

In [ ]:
# ============================================================
# 모델 저장
# ============================================================

save_dict = {
    'model_state_dict': model.state_dict(),
    'config': CONFIG,
    'threshold': threshold,
    'n_features': n_features,
    'feature_columns': feature_columns,
    'metrics': {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc_roc': auc_roc
    },
    'scaler_min': scaler.data_min_,
    'scaler_max': scaler.data_max_
}

torch.save(save_dict, 'lstm_ae_swat_model.pth')

print("✅ 모델이 'lstm_ae_swat_model.pth'로 저장되었습니다.")
print(f"\n📦 저장된 내용:")
for key in save_dict.keys():
    print(f"   - {key}")

---
## 📝 최종 결과 요약

In [ ]:
# ============================================================
# 최종 결과 요약
# ============================================================

print("=" * 60)
print("📊 SWaT LSTM-AE 이상 탐지 - 최종 결과 요약")
print("=" * 60)

print(f"\n🔧 데이터 설정:")
print(f"   - 다운샘플링: 1/{CONFIG['downsample_rate']}")
print(f"   - 피처 수: {n_features}")
print(f"   - 윈도우 크기: {CONFIG['window_size']}")
print(f"   - Stride: {CONFIG['stride']}")

print(f"\n🏗️ 모델 설정:")
print(f"   - Hidden Dim: {CONFIG['hidden_dim']}")
print(f"   - Latent Dim: {CONFIG['latent_dim']}")
print(f"   - Num Layers: {CONFIG['num_layers']}")
print(f"   - Dropout: {CONFIG['dropout']}")
print(f"   - Bidirectional: {CONFIG['bidirectional']}")

print(f"\n📈 학습 결과:")
print(f"   - 최종 학습 손실: {train_losses[-1]:.6f}")
print(f"   - 최종 검증 손실: {val_losses[-1]:.6f}")
print(f"   - Best 검증 손실: {best_val_loss:.6f}")

print(f"\n🎯 이상 탐지 성능:")
print(f"   - Threshold: {threshold:.6f}")
print(f"   - Precision: {precision:.4f}")
print(f"   - Recall: {recall:.4f}")
print(f"   - F1-Score: {f1:.4f}")
print(f"   - AUC-ROC: {auc_roc:.4f}")

print("\n" + "=" * 60)
print("✅ 완료!")
print("=" * 60)

---
## 🔄 성능 개선 방안

성능이 만족스럽지 않은 경우 다음을 시도해 보세요:

### 하이퍼파라미터 튜닝
1. `window_size` 조정: 50, 100, 200 등
2. `threshold_percentile` 조정: 95, 97, 99 등
3. `hidden_dim` 조정: 64, 128, 256 등
4. `latent_dim` 조정: 32, 64, 128 등
5. `num_layers` 조정: 1, 2, 3 등
6. `bidirectional` 활성화: 양방향 LSTM 사용

### 데이터 전처리
1. 다운샘플링 비율 조정
2. 특정 센서만 선택
3. 이동 평균 적용

### 모델 아키텍처 변형
1. GRU-AE 시도 (더 빠른 학습)
2. Attention LSTM-AE 시도
3. Variational LSTM-AE (LSTM-VAE) 시도

---
## 📚 TCN-AE vs LSTM-AE 비교

| 특성 | TCN-AE | LSTM-AE |
|------|--------|----------|
| **병렬화** | 높음 (CNN 기반) | 낮음 (순차 처리) |
| **학습 속도** | 빠름 | 느림 |
| **장기 의존성** | Dilation으로 확장 | 게이트 메커니즘 |
| **메모리 사용** | 적음 | 많음 |
| **그래디언트 흐름** | 안정적 | 소실/폭발 가능 |
| **해석 가능성** | 수용 영역 명확 | Hidden State 분석 가능 |